In [55]:
import os, sys
os.environ['PYTHONPATH'] = '/home/qihao/CS6207'
sys.path.append('/home/qihao/CS6207')
from utils.prosody_utils import *
from nltk.translate.bleu_score import sentence_bleu

In [56]:
gen_dir = 'melody_20240405:203238'
gen_date = gen_dir.split('_')[-1]
gen_melody_dir = f'/home/qihao/CS6207/output_melody/melody_{gen_date}'
gt_melody_dir = f'/home/qihao/CS6207/output_melody/melody_gt_{gen_date}'
binary_dir = f'/home/qihao/CS6207/binary'

In [57]:
gen_melodies = glob.glob(os.path.join(gen_melody_dir, '*.mid'))

In [58]:
event2word_dict, word2event_dict = pickle.load(open(f"{binary_dir}/music_dict.pkl", 'rb'))
test_data = np.load(f"{binary_dir}/words/test_words.npy", allow_pickle=True)

In [59]:
"<hi ".strip().replace('<', '')

'hi'

In [60]:
for k, item in enumerate(test_data):
    if k > 2:
        break
    src_words, tgt_words = item['src_words'], item['tgt_words']
    print(item['input_path'])
    # assert len(src_words) == len(tgt_words[1:-1])
    for idx in range(len(src_words)):
        prosody_type = word2event_dict['Prosody'][src_words[idx]['strength']]
        print(prosody_type, end='  ')
    print('\n')
    for idx in range(len(tgt_words)):
        cur_token = word2event_dict['Token'][tgt_words[idx]['token']]
        print(cur_token, end='  ')
    '''
    for idx in range(len(src_words)):
        strength = word2event_dict['Strength'][src_words[idx]['strength']]
        length = word2event_dict['Length'][src_words[idx]['length']]
        phrase = word2event_dict['Phrase'][src_words[idx]['phrase']]
        bar = word2event_dict['Bar'][tgt_words[1:-1][idx]['bar']]
        pos = word2event_dict['Pos'][tgt_words[1:-1][idx]['pos']]
        token = word2event_dict['Pitch'][tgt_words[1:-1][idx]['token']]
        dur = word2event_dict['Dur'][tgt_words[1:-1][idx]['dur']]
        # print(f"{strength}_{length}_{phrase}; {bar}_{pos}_{token}_{dur}")
    '''
    print('\n\n')

/home/qihao/CS6207/dataset/test/没有你陪伴我真的好孤单_seg0_1_Seg1.mid
<s>  <sent>  Pitch_69  <strong>  <short>  <substrong>  <short>  <weak>  <short>  <strong>  <short>  <substrong>  <short>  Pitch_72  <strong>  <short>  <weak>  <short>  <substrong>  <short>  <weak>  <short>  Pitch_69  <strong>  <short>  <sent>  Pitch_67  <strong>  <short>  <weak>  <short>  <substrong>  <short>  <weak>  <short>  Pitch_72  <strong>  <short>  <substrong>  <short>  <weak>  <short>  Pitch_69  <strong>  <short>  <strong>  <long>  <sent>  Pitch_74  <strong>  <short>  <weak>  <short>  <substrong>  <short>  <weak>  <short>  Pitch_76  <substrong>  <short>  <substrong>  <short>  <substrong>  <short>  Pitch_69  <strong>  <short>  <substrong>  <short>  <strong>  <short>  <substrong>  <short>  Pitch_74  <strong>  <short>  <sent>  Pitch_72  <strong>  <short>  <substrong>  <short>  <substrong>  <short>  <weak>  <short>  Pitch_69  <substrong>  <long>  <sent>  Pitch_67  <strong>  <short>  <substrong>  <short>  <strong>  <short> 

In [66]:
matched = 0
prsd_match = 0
prsd_all = 0

p_tknzr = {
    '<strong>': 0,
    '<substrong>': 1,
    '<weak>': 2,
    '<long>': 3,
    '<short>': 4,
}

for idx, gen_melody in enumerate(gen_melodies):
    gt_melody = os.path.join(gt_melody_dir, os.path.basename(gen_melody))
    prsd_gen = prosody(gen_melody)
    prsd_gt = prosody(gt_melody)
    # if len(prsd_gen) == len(prsd_gt):
    # print(f"Data {idx}: {os.path.basename(gen_melody)}")
    '''
    for i in range(min(len(prsd_gen), len(prsd_gt))):
        print(f"{prsd_gen[i][0]}, {prsd_gen[i][1]}, {prsd_gt[i][0]}, {prsd_gt[i][1]}")
        print('\n')
    '''

In [63]:
matched = 0
prsd_match = 0
prsd_all = 0

p_tknzr = {
    '<strong>': 0,
    '<substrong>': 1,
    '<weak>': 2,
    '<long>': 3,
    '<short>': 4,
}

for idx, gen_melody in enumerate(gen_melodies):
    gt_melody = os.path.join(gt_melody_dir, os.path.basename(gen_melody))
    prsd_gen = prosody(gen_melody)
    prsd_gt = prosody(gt_melody)
    # if len(prsd_gen) == len(prsd_gt):
    # print(f"Data {idx}: {os.path.basename(gen_melody)}")
    
    '''
    for i in range(min(len(prsd_gen), len(prsd_gt))):
        print(f"{prsd_gen[i][0]}, {prsd_gen[i][1]} \t {prsd_gt[i][0]}, {prsd_gt[i][1]}")
    print('\n')
    '''
    
    '''print(abs(len(prsd_gen)-len(prsd_gt)))'''
    
    

    if abs(len(prsd_gen)-len(prsd_gt)) < 4:
        matched += 1
    if abs(len(prsd_gen)-len(prsd_gt)) == 0:
        gt_prsd = []
        gen_prsd = []
        for i in range(len(prsd_gt)):
            gt_prsd.extend([p_tknzr[prsd_gt[i][1]]])
            gen_prsd.extend([p_tknzr[prsd_gen[i][1]]])
            # bleu_score = sentence_bleu()
            prsd_all += 1
            if prsd_gen[i][0] == prsd_gt[i][0] and prsd_gen[i][1] == prsd_gt[i][1]:
                prsd_match += 1
            # print(f"{prsd_gen[i][0]}, {prsd_gen[i][1]}, {prsd_gt[i][0]}, {prsd_gt[i][1]}")
        # print(f"|>> GEN:{gen_prsd}\n|>> GT: {gt_prsd}\n")
        bleu = sentence_bleu([gt_prsd], gen_prsd)
        # print(bleu)
        if bleu == 1.0:
            # print(f"|>> GEN:{prsd_gen}\n|>> GT: {prsd_gt}\n")
            # print(f"|>> {gen_melody}")
            pass
    
print(f"match rate {matched/len(gen_melodies)*100}%")
print(f"prosody match rate {prsd_match/prsd_all*100}%")

match rate 0.0%


ZeroDivisionError: division by zero